**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## <font color='blue'> Q3 [15 points total] </font>

### <font color='blue'> Implement a C-W MAC based on a block cipher B and a UH P based on a polynomial modulo the prime number (2^66)-5. Use XOR to combine B and P in the C-W construction. For B use the Speck cipher with a 64-bit block size and a 128-bit key size (example library implementation https://pypi.org/project/simonspeckciphers). For P, the n-th block of the message is multiplied with the n-th power of K; you may use Horner's rule to compute the polynomial iteratively and reduce to prime (2^66)-5 in each iteration to avoid operations over very large values. Showcase your program by computing the C-W MAC of message "The Carter and Wegman MAC combines a universal hash with a PRF".

In [1]:
# !pip install simonspeckciphers
from secrets import token_bytes
from speck import SpeckCipher

In [2]:
PRIME_MODULUS = (2 ** 66) - 5
HASH_SIZE = (66-1)//8 + 1
BLOCK_SIZE = 16

In [4]:
# 2 points
def bytes_to_int(b : bytes) -> int:
    """ Use little endian when converting bytes to ints """
    assert len(b) <= BLOCK_SIZE
    retval = 0
    # YOUR CODE HERE
    retval=int.from_bytes(b,'little') # bytes to int

    return retval
    
# 2 points
def bytes_to_int_blocks(msg: bytes) -> list[int]:
    """ Convert the message into an array of intgers in the "result" variable """
    """ For padding, call the byte_to_int method on the remaining message data """
    num_blocks = len(msg) // 16 #blocks
    remainder = len(msg)%BLOCK_SIZE # remaining block
    result = [0]*(num_blocks+int(remainder!=0))
    # YOUR CODE HERE
    result = [bytes_to_int(msg[i * BLOCK_SIZE:(i + 1) * BLOCK_SIZE]) for i in range(num_blocks)] # convert to int
    if remainder:
        last_block=msg[num_blocks*BLOCK_SIZE:]
        result.append(bytes_to_int(last_block))
    
    return result

# 2 points
def p_function(msg: list[int], num_blocks: int, key1: int) -> int:
    """ Implement P such that the n-th block of the message is multiplied with the n-th power of K"""
    """    result = msg[3]*key ^ 3 + msg[2]*key ^ 2 + msg[1]*key ^ 1 + msg[0]*key ^ 0 MODULO (2**66-5) """
    result = 0
    # YOUR CODE HERE
    for i in range(num_blocks):
        result=(result + (msg[i]*pow(key1,i,PRIME_MODULUS)))%PRIME_MODULUS 

    return result

# Function shouldn't be modified. Use this function in cw_mac().
# 4 points to test the correctness of previous functions.
def universal_hash(k1: bytes, msg: bytes):
    """ Universal hash implementation, using the P function """
    k1_int = bytes_to_int(k1)
    msg_int_blocks = bytes_to_int_blocks(msg)
    retval = p_function(msg_int_blocks, len(msg_int_blocks), k1_int)
    return retval
    
# 5 points
def cw_mac(k1: bytes, k2: bytes, nonce: bytes, message: bytes) -> str:
    """ Using the universal_hash() function and the SpeckCipher library,
    build the carter wegmans hash. Output should be in hex 
    string format."""
    result = 0
    assert len(k1) == 16
    assert len(k2) == 16
    # YOUR CODE HERE
    hash_value = universal_hash(k1, message) # hash
    cipher = SpeckCipher(bytes_to_int(k2), 128, 64) # initiliase cipher
    nonce_int = bytes_to_int(nonce) # nonce to int
    encrypted_nonce = cipher.encrypt(nonce_int) # encrypt nonce
    result = (hash_value + encrypted_nonce)%PRIME_MODULUS # combine with hash
    return result.to_bytes(HASH_SIZE, 'little').hex()

In [5]:
val_bytes = b"CryptoWorld!!!!!"
val_int = bytes_to_int(val_bytes)
assert val_int == 44036541601614566626012098531396514371

In [6]:
val_bytes = b"CryptoWorld!"
val_int = bytes_to_int(val_bytes)
assert val_int == 10334410032597828317038080579

In [7]:
msg = b"PitPendulumColumnatedRuinsDomino"
msg_int_arr = bytes_to_int_blocks(msg)
assert len(msg_int_arr) == 2
assert msg_int_arr[0] == 145495549588435461532754274102847760720
assert msg_int_arr[1] == 148117598509878178566457796511998959982

In [8]:
msg = b"PitPendulumColumnatedRuinsDomino!"
msg_int_arr = bytes_to_int_blocks(msg)
assert len(msg_int_arr) == 3
assert msg_int_arr[0] == 145495549588435461532754274102847760720
assert msg_int_arr[1] == 148117598509878178566457796511998959982
assert msg_int_arr[2] == 33

In [9]:
msg_int_blocks = [145495549588435461532754274102847760720, 148117598509878178566457796511998959982, 33]
k1_int = 123456789
pf_val = p_function(msg_int_blocks, len(msg_int_blocks), k1_int)
assert pf_val == 58747883318317996999

In [10]:
key1 = b"This is a secret"
val_bytes = b"CryptoWorld!!!!!"
uh_val = universal_hash(key1, val_bytes)
assert uh_val == 47900524084073003471

In [11]:
key1 = b"This is a secret"
val_bytes = b"PitPendulumColumnatedRuinsDomino!"
uh_val = universal_hash(key1, val_bytes)
assert uh_val == 63427684064651328874

In [12]:
key1 = b"This is a secret"
key2 =  b"Is this a secret"
msg = b"The Carter and Wegman MAC combines a universal hash with a PRF"
nonce = b"This is a nonce"
hashval = cw_mac(key1, key2, nonce, msg)
assert hashval == "b41d21b4c968c88f01"